In [1]:
# import pandas
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.lda import LDA
from sklearn.qda import QDA
import tensorflow as tf

import pandas 
import pandas as pd
from sklearn import preprocessing
from tensorflow.python.ops import rnn, rnn_cell

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib
tf.logging.set_verbosity(tf.logging.INFO)

#Chaikin Oscillator  
def Chaikin(df):  
    ad = (2 * df['ClosePrice'] - df['HighPrice'] - df['LowPrice']) / (df['HighPrice'] - df['LowPrice']) * df['TotalVolume']  
    Chaikin = pd.Series(pd.ewma(ad, span = 3, min_periods = 2) - pd.ewma(ad, span = 10, min_periods = 9), name = 'Chaikin')  
    df = df.join(Chaikin)  
    return df
## Bolinger Bands
def bbands(price, length=30, numsd=2):
    """ returns average, upper band, and lower band"""
    ave = pd.stats.moments.rolling_mean(price,length)
    sd = pd.stats.moments.rolling_std(price,length)
    upband = ave + (sd*numsd)
    dnband = ave - (sd*numsd)
    return np.round(ave,3), np.round(upband,3), np.round(dnband,3)

#Create output labels
def label(df, numberOfRowsAhead, priceDiffRange):
    # Create a list to store the data
    labelPosChang = []
    labelNegChang = []
    labelNoChang = []
    for i in range(0, len(df)):
       price=df.iloc[i]['ClosePrice']
       rowAheadCount=0
       if i == len(df)-1:
         print("End  of array")  
         labelPosChang.append(0)
         labelNegChang.append(0)
         labelNoChang.append(1)
         break 
          
       for j in range(i+1,i+numberOfRowsAhead):
        if j > len(df)-1:
           print("End  of array")  
           labelPosChang.append(0)
           labelNegChang.append(0)
           labelNoChang.append(1)
           break 
        priceDiff=df.iloc[j]['ClosePrice'] - price
         
        rowAheadCount = rowAheadCount +1 
        if priceDiff > priceDiffRange:
           labelPosChang.append(1)
           labelNegChang.append(0)
           labelNoChang.append(0)
           break
           
        elif priceDiff < -1*priceDiffRange:
           labelPosChang.append(0)
           labelNegChang.append(1)
           labelNoChang.append(0)
           break 
        elif numberOfRowsAhead-1 == rowAheadCount:
           labelPosChang.append(0)
           labelNegChang.append(0)
           labelNoChang.append(1)
           break
          
    print ('Done')
    print(len(labelPosChang))
    print(len(labelNegChang))
    print(len(df))
    df['Poschg'] = labelPosChang
    df['Negchg'] = labelNegChang
    df['Nochg'] = labelNoChang
   
    return df

## Generate data in RNN format
def generateRNNInputData(df,previousrows):
       X=[]
       y=[]
       for i in range(previousrows-1, len(df)):
            ## to do check whether the right element is selected
            #print(df.iloc[i-previousrows+1:i+1][['OpenPrice','HighPrice','LowPrice','TotalVolume','priceChange','movingAverage','upperBB','lowerBB','movingAverage5','upperBB5','lowerBB5','Chaikin']].as_matrix().flatten())
            X.append(df.iloc[i-previousrows+1:i+1][['OpenPrice','HighPrice','LowPrice','TotalVolume','priceChange','movingAverage','upperBB','lowerBB','movingAverage5','upperBB5','lowerBB5','Chaikin']].as_matrix().flatten())
            y.append(df.iloc[i][['Poschg','Negchg','Nochg']].as_matrix().flatten())
            
            
       #### Data Preprocessing using MinMaxScaler
       min_max_scaler = preprocessing.MinMaxScaler()
       X = min_max_scaler.fit_transform(X)
       return X,y
            



data = pandas.read_csv('C://Users/thame_000/DailyTickSandPNov112017.csv')
data=data[data['Ticker']=='MSFT']
print(data.shape)
print(data.columns)
data=Chaikin(data)
data['priceChange'] = data['ClosePrice']- data['ClosePrice'].shift() 
data['priceChangePred'] = data['ClosePrice'].shift(-1) - data['ClosePrice']

data=label(data,5,0.02)

 
          

data['movingAverage'], data['upperBB'], data['lowerBB'] = bbands(data.ClosePrice, length=30, numsd=1)
data['movingAverage5'], data['upperBB5'], data['lowerBB5'] = bbands(data.ClosePrice, length=5, numsd=1)

data.fillna(0, inplace=True)
X,y=generateRNNInputData(data,5)

input_size=len(X[0])
input_size=60

global X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


dropout = 0.1 # Dropout, probability to keep units
def weight_variable(shape):
  #initial = tf.truncated_normal(shape, stddev=0.1)
  #initial =  tf.random_normal(shape)
  return tf.Variable(tf.truncated_normal(shape, stddev=0.1))


                        
input_size=60
x=tf.placeholder('float',[None,input_size],name="x")
y_ = tf.placeholder(tf.float32, shape=[None, 3],name="y_")
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)
dropout = 0.1 # Dropout, probability to keep units
n_classes = 3
batch_size = 128
chunk_size = 12
n_chunks = 5
rnn_size = 128
## Create LSTM cell
def lstm_cell_create():
  return tf.contrib.rnn.BasicLSTMCell(
      size, forget_bias=0.0, state_is_tuple=True, reuse=tf.get_variable_scope().reuse)
## Create RNN
def recurrent_neural_network(x):
    layer = {'weights':tf.Variable(tf.random_normal([rnn_size,n_classes])),
             'biases':tf.Variable(tf.random_normal([n_classes]))}

    #x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(0, n_chunks, x)
    with tf.variable_scope('cell_def112'):
       lstm_cell = rnn_cell.BasicLSTMCell(rnn_size,state_is_tuple=True)
    #lstm_cell=lstm_cell_create()
    with tf.variable_scope('rnn_def112'):
       outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32 )

    output = tf.matmul(outputs[-1],layer['weights']) + layer['biases']

    return output
## Train neural network
def train_neural_network(x, keep_prob,input_size):
    prediction=recurrent_neural_network(x)
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(prediction,y_))
    optimizer=tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs=200
    
    countMin = 0 
    minLossEpoch = 1000
    dropout=0.1
    correct=tf.equal(tf.argmax(prediction,1),tf.argmax(y_,1))
    accuracy=tf.reduce_mean(tf.cast(correct,'float'))
    maxAccuracy=0;
    tf.scalar_summary("trainaccuracy", accuracy)
       
    # merge all summaries into a single "operation" which we can execute in a session 
    summary_op = tf.merge_all_summaries()
        
    test_summary=tf.summary.scalar("testaccuracy", accuracy)
    logs_path="C:/Users/thame_000/tbrnn"
    with tf.Session() as sess:
       sess.run(tf.initialize_all_variables())
       writer = tf.train.SummaryWriter(logs_path, graph=tf.get_default_graph())
       for epoch in range(hm_epochs):
           epoch_loss=0
           batchSize=5
           batchStartIndex= 0
           batchEndIndex= batchSize
           batchCount=0
           lengthOfTrainingSet=len(X_train)
           #print("Length training set",lengthOfTrainingSet)
           while batchEndIndex <= lengthOfTrainingSet:
               batchCount=batchCount+1
               X_train_batch=X_train[batchStartIndex:batchEndIndex]
               y_train_batch=y_train[batchStartIndex:batchEndIndex]
               _,c,summary=sess.run([optimizer,cost, summary_op],feed_dict={x:X_train_batch,y_:y_train_batch, keep_prob: dropout})
               #print('Epoch',epoch,'completed out of',hm_epochs,'loss_epoch:',c)
               valid_acc, valid_summ = sess.run(
                            [accuracy, test_summary],
                                feed_dict={x:X_test,y_:y_test, keep_prob: dropout})
               currAccuracy=valid_acc
               writer.add_summary(valid_summ, epoch *  batchSize + batchCount)
               #print('Accuracy:',currAccuracy)
               if epoch == 0:
                  maxAccuracy=currAccuracy
               if currAccuracy >  maxAccuracy:
                  maxAccuracy=currAccuracy
                  print('maxAccuracy:',maxAccuracy)
               #if countMin > 100 :
                 #break  q
               if batchEndIndex >lengthOfTrainingSet-1 :
                 break
               batchStartIndex=batchEndIndex
               batchEndIndex=batchEndIndex+batchSize
               if batchEndIndex >lengthOfTrainingSet-1 :
                  batchEndIndex=lengthOfTrainingSet
               writer.add_summary(summary, epoch *  batchSize + batchCount)
       print('maxAccuracy',maxAccuracy)




batch_size=50
dropout = 0.1 # Dropout, probability to keep units

#12 is number of columns
numofinput=12

train_neural_network(x, keep_prob,numofinput)

C:\Users\thame_000\Anaconda3new\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
C:\Users\thame_000\Anaconda3new\lib\site-packages\sklearn\qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


(391, 10)
Index(['Date', 'Timestamp', 'Ticker', 'OpenPrice', 'HighPrice', 'LowPrice',
       'ClosePrice', 'TotalVolume', 'TotalQuantity', 'TotalTradeCount'],
      dtype='object')


C:\Users\thame_000\Anaconda3new\lib\site-packages\ipykernel\__main__.py:21: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(min_periods=2,ignore_na=False,span=3,adjust=True).mean()
C:\Users\thame_000\Anaconda3new\lib\site-packages\ipykernel\__main__.py:21: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(min_periods=9,ignore_na=False,span=10,adjust=True).mean()


End  of array
End  of array
End  of array
End  of array
Done
391
391
391


C:\Users\thame_000\Anaconda3new\lib\site-packages\ipykernel\__main__.py:27: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=30).mean()
C:\Users\thame_000\Anaconda3new\lib\site-packages\ipykernel\__main__.py:28: FutureWarning: pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=30).std()
C:\Users\thame_000\Anaconda3new\lib\site-packages\ipykernel\__main__.py:27: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=5).mean()
C:\Users\thame_000\Anaconda3new\lib\site-packages\ipykernel\__main__.py:28: FutureWarning: pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(center=False,window=5).std()


6880.0
8602
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updating:
Please switch to tf.summary.merge_all.
Instructions for updating:
Please switch to tf.summary.merge.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.
maxAccuracy: 0.410256
maxAccuracy: 0.423077
maxAccuracy: 0.487179
maxAccuracy: 0.5
maxAccuracy: 0.512821
maxAccuracy: 0.525641
maxAccuracy: 0.538462
maxAccuracy: 0.551282
maxAccuracy: 0.564103
maxAccuracy: 0.576923
maxAccuracy 0.576923
